Testing filtering of stop words in dataset. To do this we can use the python natural language toolkit: https://www.nltk.org/

In [9]:
import nltk
from nltk.corpus import stopwords
import string
nltk.download('stopwords')
nltk.download('punkt_tab')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /home/jikael/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jikael/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [25]:
# from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer # use this one because the other one struggles with words like "don't"

tokenizer = TweetTokenizer()
 
example_sent = """This is a sample sentence,
                  showing off the stop words filtration. Don't."""

stop_words = set(stopwords.words('english'))
 
word_tokens = tokenizer.tokenize(example_sent)
# converts the words in word_tokens to lower case and then checks whether 
#they are present in stop_words or not
filtered_sentence = ' '.join([w for w in word_tokens if (not w.lower() in stop_words) and (not w in string.punctuation)])

print(word_tokens)
print(filtered_sentence)

['This', 'is', 'a', 'sample', 'sentence', ',', 'showing', 'off', 'the', 'stop', 'words', 'filtration', '.', "Don't", '.']
sample sentence showing stop words filtration


In [ ]:
def filter_sentence(sentence, stop_words, tokenizer):
    word_tokens = tokenizer.tokenize(sentence)
    filtered_sentence = ' '.join([w for w in word_tokens if (not w.lower() in stop_words) and (not w in string.punctuation)])
    return filtered_sentence

SyntaxError: invalid syntax (1199914503.py, line 3)

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
    "What if I use don't?"
]
filtered_corpus = [filter_sentence(sentence, stop_words, tokenizer) for sentence in corpus]
# filtered_corpus
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(filtered_corpus)
vectorizer.get_feature_names_out()
# print(X.shape)

array(['document', 'first', 'one', 'second', 'third', 'use'], dtype=object)

In [81]:
print(X.toarray())

[[0.63871058 0.76944707 0.         0.         0.         0.        ]
 [0.80130969 0.         0.         0.59824977 0.         0.        ]
 [0.         0.         0.70710678 0.         0.70710678 0.        ]
 [0.63871058 0.76944707 0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         1.        ]]


In [82]:
import pandas as pd
df = pd.read_csv('../data/vance.tsv', sep='\t')
df['valid'] = df['description'].map(lambda x: type(x) == str)
df=df[df['valid']]

In [85]:
df.head()

,Unnamed: 0,source,author,title,description,url,publishedAt,topic,negative/neutral/positive,valid,filtered_description
0,2,Biztoc.com,breakingthenews.net,"JD Vance slams Harris: 'Tomorrow, you're fired'",Republican vice presidential candidate JD Vanc...,https://biztoc.com/x/c4c538f75498736e,2024-11-04T22:53:56Z,NaN,NaN,True,republican vice presidential candidate jd vanc...
1,3,Biztoc.com,nbcnews.com,'Feels kind of weird': Vance reflects and look...,"Sen. JD Vance, who says running as Trump's VP ...",https://biztoc.com/x/339813fc9b9bc9be,2024-11-04T22:53:24Z,NaN,NaN,True,sen jd vance says running trump's vp pick cost...
2,6,NBC News,Henry J. Gomez and Alec Hernández,'Feels kind of weird': Vance reflects and look...,"Sen. JD Vance, who says running as Trump's VP ...",https://www.nbcnews.com/politics/2024-election...,2024-11-04T22:15:42Z,NaN,NaN,True,sen jd vance says running trump's vp pick cost...
3,7,Bleeding Cool News,Ray Flook,Last Week Tonight: HBO Releases John Oliver Se...,"For us, Sunday night's edition of HBO's Last W...",https://bleedingcool.com/tv/last-week-tonight-...,2024-11-04T15:07:08Z,NaN,NaN,True,us sunday night's edition hbo's last week toni...
4,10,New York Post,Emily Crane,JD Vance says Trump is ‘fired up’ over death o...,Republican vice presidential candidate JD Vanc...,https://nypost.com/2024/11/04/us-news/jd-vance...,2024-11-04T15:06:04Z,NaN,NaN,True,republican vice presidential candidate jd vanc...


In [83]:
df['filtered_description'] = df['description'].map(lambda x: filter_sentence(x, stop_words, tokenizer))

In [87]:
def get_filtered_descriptions(df, stop_words, tokenizer):
    df['filtered_description'] = df['description'].map(lambda x: filter_sentence(x, stop_words, tokenizer))

In [88]:
get_filtered_descriptions(df, stop_words, tokenizer)

In [89]:
df.head()

,Unnamed: 0,source,author,title,description,url,publishedAt,topic,negative/neutral/positive,valid,filtered_description
0,2,Biztoc.com,breakingthenews.net,"JD Vance slams Harris: 'Tomorrow, you're fired'",Republican vice presidential candidate JD Vanc...,https://biztoc.com/x/c4c538f75498736e,2024-11-04T22:53:56Z,NaN,NaN,True,republican vice presidential candidate jd vanc...
1,3,Biztoc.com,nbcnews.com,'Feels kind of weird': Vance reflects and look...,"Sen. JD Vance, who says running as Trump's VP ...",https://biztoc.com/x/339813fc9b9bc9be,2024-11-04T22:53:24Z,NaN,NaN,True,sen jd vance says running trump's vp pick cost...
2,6,NBC News,Henry J. Gomez and Alec Hernández,'Feels kind of weird': Vance reflects and look...,"Sen. JD Vance, who says running as Trump's VP ...",https://www.nbcnews.com/politics/2024-election...,2024-11-04T22:15:42Z,NaN,NaN,True,sen jd vance says running trump's vp pick cost...
3,7,Bleeding Cool News,Ray Flook,Last Week Tonight: HBO Releases John Oliver Se...,"For us, Sunday night's edition of HBO's Last W...",https://bleedingcool.com/tv/last-week-tonight-...,2024-11-04T15:07:08Z,NaN,NaN,True,us sunday night's edition hbo's last week toni...
4,10,New York Post,Emily Crane,JD Vance says Trump is ‘fired up’ over death o...,Republican vice presidential candidate JD Vanc...,https://nypost.com/2024/11/04/us-news/jd-vance...,2024-11-04T15:06:04Z,NaN,NaN,True,republican vice presidential candidate jd vanc...
